In [ ]:
# default_exp core

In [ ]:
#export
import matplotlib.pyplot as plt
from medtop.nlp_helpers import *
from medtop.preprocessing import *
import numpy
import pandas as pd
import plotly
import plotly.express as px
from scipy.cluster.hierarchy import ward, cut_tree
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.manifold import MDS
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
import os
from sklearn.decomposition import TruncatedSVD
import umap.umap_ as umap

# Core

> Methods for calling 

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def import_docs(path_to_file_list, verbose = False):
    "Imports and processes the list of documents contained in the given file"
    
    # Extract list of files from text document
    with open(path_to_file_list) as file:
        file_list = file.read().strip().split('\n')
    
    # Append the raw contents of each document to an array
    file_content = []
    for file_path in file_list:
        with open(file_path, "r") as file:
            text = file.read()
            file_content.append(text)
        
    my_docs = []
    my_docs_pos = []
    my_docs_loc = []    
    raw_docs = []
    raw_sentences = []

    for doc in file_content:
        split_sent, split_sent_pos, split_sent_loc, tokens, pos, loc, full_sent, full_pos, full_loc, raw_sent = tokenize_and_stem(doc)
        my_docs.append(split_sent)
        my_docs_pos.append(split_sent_pos)
        my_docs_loc.append(split_sent_loc)
        raw_docs.append(doc)
        raw_sentences.append(raw_sent)
        
    if verbose == True: print("Number of Documents Loaded: " + str(len(my_docs)))
        
    return my_docs, my_docs_pos, my_docs_loc, raw_sentences, raw_docs

In [ ]:
#export
def sentences_to_disk(raw_sentences, outfile_name):
    "Writes the raw sentences to a file organized by document and sentence number"
#     outfile_name = args.o + "/" + args.p + "_DocumentSentenceList.txt"
    with open(outfile_name, "w") as outfile:
        outfile.write("Sentence ID\tSentence Text\n")
        for doc_ix, doc in enumerate(raw_sentences):
            for sent_ix, sent in enumerate(doc):
                outfile.write(f"doc.{doc_ix}.sent.{sent_ix}\t{sent}\n")

In [ ]:
#export
def get_doc_top_phrases(raw_docs, feature_names, tdm, window_size = 6, include_input_in_tfidf = False):
    #TODO: Do this without reparsing each sentence in get_top_phrases
    doc_top_phrases = []    
    for doc_id, text in enumerate(raw_docs):
        top_phrases = get_top_phrases(doc_id, text, feature_names, tdm, window_size, include_input_in_tfidf)
        doc_top_phrases.append(top_phrases)
        
    return doc_top_phrases

In [ ]:
#export
def get_doc_word_vectors_tfidf(doc_top_phrases, dictionary, tfidf_dense):
    "Uses the TF-IDF matrix to create a list of lists corresponding to a word vectors for the top phrases of each sentence in each document"
    return get_phrase_vec_tfidf(doc_top_phrases, dictionary, tfidf_dense)

def get_doc_word_vectors_svd(doc_top_phrases, dictionary, tfidf_dense, dimensions = 2):
    "Decomposes the TF-IDF matrix via SVD and uses the result to create a list of lists corresponding to a word vectors for the top phrases of each sentence in each document"
    svd =  TruncatedSVD(n_components = dimensions, random_state = 42)
    tfidf_transf = svd.fit_transform(tfidf_dense)
    return get_phrase_vec_tfidf(doc_top_phrases, dictionary, tfidf_transf)

def get_doc_word_vectors_umap(doc_top_phrases, dictionary, tfidf_dense, umap_neighbors = 15, dimensions = 2):
    "Transforms the TF-IDF matrix via UMAP and uses the result to create a list of lists corresponding to a word vectors for the top phrases of each sentence in each document"
    reducer = umap.UMAP(n_neighbors=umap_neighbors, min_dist=.1, metric='cosine', random_state=42, n_components = dimensions)
    embed = reducer.fit_transform(tfidf_dense)
    return get_phrase_vec_tfidf(doc_top_phrases, dictionary, embed)

def get_doc_word_vectors_pretrained(doc_top_phrases, path_to_w2v_bin_file):
    "Uses pretrained Word2Vec embeddings to create a list of lists corresponding to a word vectors for the top phrases of each sentence in each document"
    assert os.path.exists(path_to_w2v_bin_file), f'The file {path_to_w2v_bin_file} does not exist'
    model = w2v_pretrained(path_to_w2v_bin_file)
    return get_phrase_vec_w2v(doc_top_phrases, model)

def get_doc_word_vectors_local(doc_top_phrases, raw_docs, my_docs):
    "Creates Word2Vec embeddings from the input corpus and uses them to create a list of lists corresponding to a word vectors for the top phrases of each sentence in each document"
    model = w2v_from_corpus(raw_docs, my_docs)
    return get_phrase_vec_w2v(doc_top_phrases, model)

In [ ]:
#export
def filter_sentences(doc_phrase_vecs, doc_top_phrases):
    "Filter the sentences, removing any that contain zero 'top phrases'"
    just_phrase_vecs = []
    just_phrase_ids = []
    just_phrase_text = []

    # Getting all phrases which have 
    # Iterate documents
    for doc_ix, phrase_vecs in enumerate(doc_phrase_vecs):
        # Iterate sentences (each sentence has a phrase vector)
        for phrase_ix, phrase_vec in enumerate(phrase_vecs):
            phrase_id = f"doc.{doc_ix}.sent.{phrase_ix}"
            this_text = doc_top_phrases[doc_ix][phrase_ix]

            # Check phrase_vec is not zero or an array of zeros, these would imply the sentence has no "top phrases"
            if isinstance(phrase_vec, numpy.ndarray) and any(numpy.zeros(len(phrase_vec)) != phrase_vec):
                just_phrase_vecs.append(list(phrase_vec))
                just_phrase_ids.append(phrase_id)
                just_phrase_text.append(this_text[1])
    return just_phrase_vecs, just_phrase_ids, just_phrase_text

In [ ]:
#export 
def get_optimal_k(just_phrase_vecs, save_chart = False, chart_file = "KmeansSilhouette.png"):
    """
    Clusters the top phrase vectors for a range of k values and plots the silhoute coefficients 
    (this is a function of mean intra-cluster distance and mean nearest-cluster distance). 
    Returns the optimal k-value (highest silhoute coefficient)
    """
    max_k = min(len(just_phrase_vecs), 100)
    k_range = range(2, max_k)
    score = [(silhouette_score(just_phrase_vecs, KMeans(i).fit(just_phrase_vecs).predict(just_phrase_vecs))) for i in k_range]
    fig = plt.plot(k_range, score)
    if save_chart:
        plt.savefig(chart_file, dpi=300)
    optimal_k = k_range[numpy.argmax(score)]
    return optimal_k

def get_cluster_assignments_kmeans(k, just_phrase_vecs):
    "Use K-means algorithm to cluster phrase vectors"
    kmeans = KMeans(n_clusters=k, random_state=42).fit(just_phrase_vecs)
    cluster_assignments = kmeans.predict(just_phrase_vecs)
    #TODO: this is never used before being overwritten later. Remove?
    dist = pairwise_distances(just_phrase_vecs, metric='euclidean') 
    return cluster_assignments, dist

In [ ]:
#export
def get_linkage_matrix(just_phrase_vecs, dist_metric):
    "Creates a linkage matrix by calculating distance between phrase vectors"
    if dist_metric == "cosine":
        dist = 1 - cosine_similarity(just_phrase_vecs)
    else:
        dist = pairwise_distances(just_phrase_vecs, metric=dist_metric)
    return ward(dist)

def get_optimal_height(just_phrase_vecs, linkage_matrix, save_chart = False, chart_file = "HACSilhouette.png"):
    """
    Clusters the top phrase vectors and plots the silhoute coefficients for a range of dendrograph heights. 
    Returns the optimal height value (highest silhoute coefficient)
    """
    max_h = min(len(just_phrase_vecs), 100)
    # QUESTION: What is the limiting factor for the min/max in this range?
#     h_range = range(30, max_h) 
    h_range = range(1,10) 
    h_score = [(silhouette_score(just_phrase_vecs, [x[0] for x in cut_tree(linkage_matrix, height=i)] )) for i in h_range]

    fig = plt.plot(h_range, h_score)

    if save_chart:
        plt.savefig(chart_file, dpi=300)
    optimal_h = h_range[numpy.argmax(h_score)]
    return optimal_h

def get_cluster_assignments_hac(linkage_matrix, height):
    "Use Hierarchical Agglomerative Clustering to cluster phrase vectors"
    return [x[0] for x in cut_tree(linkage_matrix, height=height)]

In [ ]:
#export
def visualize_umap(just_phrase_ids, cluster_assignments, just_phrase_text, dist, umap_neighbors = 15, save_chart = False, chart_file = "UMAP.html"):
    "Visualize the clusters using UMAP"
    reducer = umap.UMAP(n_neighbors=umap_neighbors, min_dist=.1, metric='cosine', random_state=42)
    embedding = reducer.fit_transform(dist)
    df = pd.DataFrame(dict(label=just_phrase_ids, UMAP_cluster=cluster_assignments, phrase=just_phrase_text, x=embedding[:, 0], y=embedding[:, 1]))
    fig = px.scatter(df, x="x", y="y", hover_name="label", color="UMAP_cluster", hover_data=["phrase", "label","UMAP_cluster"], color_continuous_scale='rainbow')
    fig.show()
    if save_chart:
        plotly.offline.plot(fig, filename=chart_file)
        
def visualize_mds(just_phrase_ids, cluster_assignments, just_phrase_text, dist, save_chart = False, chart_file = "MDS.html"):
    "Visualize the clusters using Multi-Dimensional Scaling (MDS)"
    mds = MDS(n_components=2, dissimilarity="precomputed", random_state=42)
    pos = mds.fit_transform(dist)
    xs, ys = pos[:, 0], pos[:, 1]
    df = pd.DataFrame(dict(x=xs, y=ys, MDS_cluster=cluster_assignments, label=just_phrase_ids, phrase=just_phrase_text)) 
    fig = px.scatter(df, x="x", y="y", hover_name="label", color="MDS_cluster", hover_data=["phrase", "label","MDS_cluster"], color_continuous_scale='rainbow')
    fig.show()
    if save_chart:
        plotly.offline.plot(fig, filename=chart_file)
        
def visualize_svd(just_phrase_ids, cluster_assignments, just_phrase_text, dist, save_chart = False, chart_file = "SVD.html"):
    "Visualize the clusters using Singular Value Decomposition (SVD)"
    svd2d =  TruncatedSVD(n_components = 2, random_state = 42).fit_transform(dist)
    df = pd.DataFrame(dict(label=just_phrase_ids, SVD_cluster=cluster_assignments, phrase=just_phrase_text, x=svd2d[:, 0], y=svd2d[:, 1])) 
    fig = px.scatter(df, x="x", y="y", hover_name="label", color="SVD_cluster", hover_data=["phrase", "label","SVD_cluster"], color_continuous_scale='rainbow')
    fig.show()
    if save_chart:
        plotly.offline.plot(fig, filename=chart_file)

In [ ]:
#export
def get_cluster_topics(cluster_assignments, just_phrase_ids, my_docs):
    "Get list of phrases by document/sentence"
    cluster_topics = []
    for c in set(cluster_assignments):
        #QUESTION: I removed the condition len(x.split(".")) > 1 is there a case when that wouldn't be True?
        # Parse (doc_id, sent_id) "coordinate" pairs from dataframe label column
        cluster_mask = numpy.asarray(cluster_assignments) == c
        # TODO: Consolidate the next two lines if Amy's okay with storing just_phrase_ids as a list of tuples
        topic_labels = numpy.asarray(just_phrase_ids)[cluster_mask]
        sent_coords = [[int(x.split(".")[1]), int(x.split(".")[3])] for x in topic_labels] 

        ## sent_coords has each element formatted as [doc number, sentence number]
        cluster_terms = []
        for doc_id, sent_id in sent_coords:
            sent = my_docs[doc_id][sent_id]
            # This isn't used. Can we remove?
#             pos = my_docs_pos[doc_id][sent_id] 
            cluster_terms.append(sent)

        cluster_topics.append(cluster_terms)
    return cluster_topics

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted Core.ipynb.
Converted helpers.ipynb.
This cell doesn't have an export destination and was ignored:
e
Converted index.ipynb.
Converted nlp_helpers.ipynb.
Converted preprocessing.ipynb.
Converted Sandbox.ipynb.
